In [6]:
######################################################################
# Setup python environment and change the current working directory
######################################################################
!pip3 install torch torchvision
!pip3 install Pillow==4.0.0
!pip3 install tensorboardX
%mkdir -p /content/final_project
%cd /content/final_project
%mkdir ./output
%mkdir ./data

  Using cached https://files.pythonhosted.org/packages/d2/c2/f84b1e57416755e967236468dcfb0fad7fd911f707185efc4ba8834a1a94/Pillow-6.0.0-cp36-cp36m-manylinux1_x86_64.whl
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.2.post3 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
scikit-image 0.14.2 has requirement pillow>=4.3.0, but you'll have pillow 4.0.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 6.0.0
    Uninstalling Pillow-6.0.0:
      Successfully uninstalled Pillow-6.0.0


/content/final_project
mkdir: cannot create directory ‘./output’: File exists
mkdir: cannot create directory ‘./data’: File exists


# Import Libraries and Setting Constants

In [0]:
from torch.utils.data import Dataset
from tensorboardX import SummaryWriter
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import torch
import numpy as numpy
import torch.nn.functional as F
import math
import copy
import time
from torch.autograd import Variable
from torch.optim import Adam
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

from IPython.display import HTML, display
import shutil

In [0]:
!mv /content/test_Laliga_Complete_Dataset_All.csv /content/final_project/data/

In [0]:
!mv /content/train_all.csv /content/final_project/data/

# MatchDataset

In [0]:
class MatchDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path, index_col=0)

        self.target_df = df.iloc[:, 25] # home_win = 0, away_win = 1, draw = 2
        self.baseline_df = df.iloc[:, 26:620]
    def __len__(self):
        return len(self.baseline_df.index) - 1

    def __getitem__(self, index):
        baseline_input = torch.tensor(self.baseline_df.iloc[index, :], dtype=torch.float)
        target = torch.tensor(self.target_df.iloc[index], dtype=torch.long)

        output = {'baseline_input': baseline_input, 'target':target}

        return output

In [0]:
train_dataset = MatchDataset('data/train_Laliga_Complete_Dataset_All.csv')
#train_dataset = MatchDataset('data/train_all.csv')
train_dataset.target_df

In [0]:
#validation_dataset = MatchDataset('data/test_all.csv')
validation_dataset = MatchDataset('data/test_Laliga_Complete_Dataset_All.csv')
validation_dataset.baseline_df

# BMM

In [0]:
class Fully_Connected(nn.Module):
    def __init__(self, output_size=3):
        super(Fully_Connected, self).__init__()
        self.fc = nn.Linear(729, output_size)

    def forward(self, x):
        batch_size = x.shape[0]
        x1 = x[:, 0:297]
        x2 = x[:, 297:594]
        home_team = torch.ones(batch_size, 11, 27)
        away_team = torch.ones(batch_size, 11, 27)
        for i in range(batch_size):
          for j in range(11):
            for k in range(27):
              home_team[i,j,k] = x1[i, j * k + k]
              away_team[i,j,k] = x2[i, j * k + k]
        # size of new_x is batch_size * 27 * 27
        new_x = torch.bmm(home_team.transpose(1,2), away_team)
        x_vector = new_x.view(batch_size, 729)
        x_vector = F.relu(self.fc(x_vector))
        y = F.softmax(x_vector, dim=1)
        return y

# FULLY CONNECTED LAYER TRAINER

In [0]:
class FullyConnectedTrainer:
    def __init__(self, FullyConnected, train_dataloader, val_dataloader=None, lr=1e-3, 
                 with_cuda=True, cuda_device=None):
        cuda_condition = torch.cuda.is_available() and with_cuda
        self.device = torch.device("cpu" if cuda_condition else "cpu")
#         self.device = torch.device("cuda:0" if cuda_condition else "cpu")
        self.model = Fully_Connected().to(self.device)
        self.p_loss = nn.CrossEntropyLoss()
        self.FullyConnected = FullyConnected
        
        if with_cuda and torch.cuda.device_count() > 1:
            print("Using %d GPUS for Fully Connected" % torch.cuda.device_count())
            self.model = nn.DataParallel(self.model, device_ids=cuda_devices)

        self.train_data = train_dataloader
        self.val_data = val_dataloader

        self.optimizer = torch.optim.SGD((self.model).parameters(), lr=lr, momentum=0.95)

        self.losses = {'train_loss':[], 'validation_loss':[]}
        self.accuracy = {'train_accuracy':[], 'validation_accuracy':[]}
            

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.val_data, False)

    def iteration(self, epoch, data_loader, train=True):
        """ Iteration process for one epoch. 
        """
        process = "train" if train else "validation"

        data_iterator = tqdm_notebook(enumerate(data_loader),
                              desc="EP %d %s" % (epoch, process),
                              total=len(data_loader),
                              bar_format="{l_bar}{r_bar}")

        average_loss = 0.0
        correct = 0
        total = 0
        num_0 = 0
        num_1 = 0
        num_2 = 0
        
        for i, data in data_iterator:
            data = {key: value.to(self.device) for key, value in data.items()}
            output = self.model.forward(data['baseline_input'])
            hit = output.argmax(dim=-1).eq(data["target"]).sum().item()
            miss = data["target"].nelement() - hit
            
            loss = self.p_loss(output, data['target'])

            if train:
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

            average_loss += loss.item()
            correct += hit
            total += data["target"].nelement()
            
            # count 
            num_0 += output.argmax(dim=-1).eq(0).sum().item()
            num_1 += output.argmax(dim=-1).eq(1).sum().item()
            num_2 += output.argmax(dim=-1).eq(2).sum().item()

            log_info = {
                    "epoch": epoch, 
                    "iteration": i,
                    "total_loss": average_loss / (i + 1),
                    "average_accuracy": correct / total * 100,
            }
        
        print('0:', num_0, ' | ', '1:', num_1, ' | ', '2:', num_2)
        print('average_accuracy: ', log_info['average_accuracy'])
        print('total_loss: ', log_info['total_loss'])

        self.losses['{}_loss'.format(process)].append(log_info['total_loss'])
        self.accuracy['{}_accuracy'.format(process)].append(log_info['average_accuracy'])

    def save(self, epoch, filepath = "output/trained.model"):
            if epoch % 10 == 0:
                output_path = filepath + ".ep%d" % epoch
                torch.save(self.FullyConnected.cpu(), output_path)
                self.FullyConnected.to(self.device)
                print("EP:%d Model saved to:" % epoch, output_path)
                return output_path

## Training Loop

In [0]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


def print_opts(opts):
    """Prints the values of arguments.
    """
    print('=' * 80)
    print('Opts'.center(80))
    print('-' * 80)
    for key in opts.__dict__:
        print('{:>30}: {:<30}'.format(key, str(opts.__dict__[key])).center(80))
    print('=' * 80)


def train(args):
    # Load Data
    print("=== Loading datasets ===")
    train_dataset = MatchDataset(args.train_path)
    validation_dataset = MatchDataset(args.val_path)
    train_dataloader = DataLoader(train_dataset,
                                  batch_size=args.batch_size, num_workers=10)

    val_dataloader = DataLoader(validation_dataset,
                                     batch_size=args.batch_size, num_workers=10)
    
    


    # Build model
    print("=== Building Model ===")
    baseline = Fully_Connected()

    trainer = FullyConnectedTrainer(FullyConnected=baseline, 
                          train_dataloader=train_dataloader,
                          val_dataloader=val_dataloader, 
                          lr=args.lr)

    print("=== Start Training ===")
    for epoch in range(args.epochs):
        print ("epoch number: " + str(epoch))
        trainer.train(epoch)
        trainer.save(epoch, args.output_path)

        if val_dataloader is not None:
            trainer.test(epoch)
    
    loss_df = pd.DataFrame.from_dict(trainer.losses)
    accuracy_df = pd.DataFrame.from_dict(trainer.accuracy)
    loss_df.plot(xticks=range(0, args.epochs, 5))
    accuracy_df.plot(xticks=range(0, args.epochs,5))
    
    return baseline, trainer
    

# Train

In [19]:
args = AttrDict()
default_config = {
    "train_path": "data/train_Laliga_Complete_Dataset_All.csv",
    "val_path": "data/test_Laliga_Complete_Dataset_All.csv",
    "batch_size": 100,
    "epochs": 100,
    "lr": 0.001,
    "output_path": 'output/exp1.model',
}

args.update(default_config)
print_opts(args)
baseline, trainer = train(args)

                                      Opts                                      
--------------------------------------------------------------------------------
                       train_path: data/train_Laliga_Complete_Dataset_All.csv   
                         val_path: data/test_Laliga_Complete_Dataset_All.csv    
                             batch_size: 100                                    
                                 epochs: 100                                    
                                     lr: 0.001                                  
                            output_path: output/exp1.model                      
=== Loading datasets ===
=== Building Model ===
=== Start Training ===
epoch number: 0



0: 0  |  1: 0  |  2: 1798
average_accuracy:  23.136818687430477
total_loss:  1.09861159324646
EP:0 Model saved to: output/exp1.model.ep0


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Fully_Connected. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "



0: 0  |  1: 0  |  2: 447
average_accuracy:  24.832214765100673
total_loss:  1.0986117839813232
epoch number: 1



0: 0  |  1: 0  |  2: 1798
average_accuracy:  23.136818687430477
total_loss:  1.09861159324646


...


0: 0  |  1: 0  |  2: 1798
average_accuracy:  23.136818687430477
total_loss:  1.09861159324646



0: 0  |  1: 0  |  2: 447
average_accuracy:  24.832214765100673
total_loss:  1.0986117839813232
epoch number: 32



0: 0  |  1: 0  |  2: 1798
average_accuracy:  23.136818687430477
total_loss:  1.09861159324646



0: 0  |  1: 0  |  2: 447
average_accuracy:  24.832214765100673
total_loss:  1.0986117839813232
epoch number: 33


KeyboardInterrupt: ignored